## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [2]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
import statsmodels.stats.proportion as propstest

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


# Data Explotation

In [3]:
#check missing values
train_data.isnull().sum()

ID           0
Promotion    0
purchase     0
V1           0
V2           0
V3           0
V4           0
V5           0
V6           0
V7           0
dtype: int64

In [4]:
#check statistics of training data
train_data.describe()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000
mean,62970.972413,0.012303,1.500662,29.973600,0.000190,1.679608,2.327643,2.502898,1.701694
std,36418.440539,0.110234,0.868234,5.010626,1.000485,0.466630,0.841167,1.117349,0.457517
min,1.000000,0.000000,0.000000,7.104007,-1.684550,1.000000,1.000000,1.000000,1.000000
25%,31467.250000,0.000000,1.000000,26.591501,-0.905350,1.000000,2.000000,2.000000,1.000000
50%,62827.500000,0.000000,2.000000,29.979744,-0.039572,2.000000,2.000000,3.000000,2.000000
75%,94438.750000,0.000000,2.000000,33.344593,0.826206,2.000000,3.000000,4.000000,2.000000
max,126184.000000,1.000000,3.000000,50.375913,1.691984,2.000000,4.000000,4.000000,2.000000


In [5]:
#check data type
train_data.dtypes

ID             int64
Promotion     object
purchase       int64
V1             int64
V2           float64
V3           float64
V4             int64
V5             int64
V6             int64
V7             int64
dtype: object

In [9]:
#calculate the size of each group
n_ctrl = train_data.groupby('Promotion')['Promotion'].count()[0]
n_treat = train_data.groupby('Promotion')['Promotion'].count()[1]
print ('The control group is of size %d' %n_ctrl)
print ('The treat group is of size %d' %n_treat)

The control group is of size 42170
The treat group is of size 42364


In [10]:
#calculate the number of purchaser in each group
pur_ctrl = train_data.groupby('Promotion')['purchase'].sum()[0]
pur_treat = train_data.groupby('Promotion')['purchase'].sum()[1]

print ('The number of purchaser in the control group is %d' %pur_ctrl)
print ('The number of purchaser in the treat group is %d' %pur_treat)

The number of purchaser in the control group is 319
The number of purchaser in the treat group is 721


In [11]:
pct_ctrl = n_ctrl / train_data.shape[0]
pct_treat = n_treat / train_data.shape[0]

print("proportion of no promotion is %.4f, proportion of promotion is %.4f" % (pct_ctrl, pct_treat))

proportion of no promotion is 0.4989, proportion of promotion is 0.5011


In [14]:
print("The purchase ratio of treat group is " , round(pur_treat/n_treat, 4))
print("The purchased ratio of control group is ", round(pur_ctrl/n_ctrl, 4))

The purchase ratio of treat group is  0.017
The purchased ratio of control group is  0.0076


In [15]:
# Use z_test to test if the purchase ratio of treat group is significantly higher with alpha = 0.05
count = np.array([pur_ctrl, pur_treat])
obs = np.array([n_ctrl, n_treat])

z, p = propstest.proportions_ztest(count, obs)

alpha = 0.05
if (p < alpha):
    print('The purchase ratio of treat group is significantly higher than the control group.')
else:
    print('The purchase ratio of treat group is not significantly higher than the control group.')


The purchase ratio of treat group is significantly higher than the control group.


# Define Metrics

In [16]:
# Define first evaluation metric IRR
def irr(pur_treat, n_treat, pur_ctrl, n_ctrl):
    return (pur_treat / n_treat - pur_ctrl / n_ctrl)

In [17]:
# Define second evaluation metric NIR
def nir(pur_treat, n_treat, pur_ctrl):
    return (pur_treat * 10 - n_treat * 0.15 - pur_ctrl * 10)

In [19]:
train_data_irr = irr(pur_treat, n_treat, pur_ctrl, n_ctrl)
train_data_nir = nir(pur_treat, n_treat, pur_ctrl)
print('The IRR of the train data is %.4f' %train_data_irr)
print('The NIR of the train data is %.4f' %train_data_nir)

The IRR of the train data is 0.0095
The NIR of the train data is -2334.6000


# Modeling

The optimization strategy is distributing coupons only to the people who will purchase the product. So the purpose of this model is try to predict which customer will purchase the product based on theIR features V1-V7. 
The modeling process is as following:
* Randomly extract 1040 customers from people who didn't purchase the product, and combine them with customers who bought the product, so that the two group are of the same size.
* Find categorical variables and generate dummy variables from them
* Split the total sample into train group and test group, with test_size = 0.2
* Use the train group to train the Logistic Regression model
* Use the model to predict the purchasing behavior of the test group
* Evaluate the model

In [20]:
# Randomly sample 1040 rows from not purchased dataset
# Combine it with all the rows from purchased dataset
pur = train_data[train_data['purchase'] == 1]
not_pur = train_data[train_data['purchase'] == 0].sample(1040)
sample = pd.concat([pur, not_pur], axis = 0)
sample.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
107,162,Yes,1,1,33.030562,-0.732194,2,1,1,1
328,503,Yes,1,3,31.760033,1.518828,2,1,2,2
352,534,Yes,1,2,17.512709,1.259095,2,3,4,2
383,580,Yes,1,1,31.043735,-1.338239,2,1,2,1
423,639,No,1,1,34.901993,-0.905350,2,3,2,2


In [21]:
# Get categorical variables and generate dummy variables
cat_var = train_data.select_dtypes('int').columns
cat_var = cat_var.drop(['ID', 'purchase'])

sample = pd.get_dummies(data = sample, columns = cat_var)
sample.head()

,ID,Promotion,purchase,V2,V3,V1_0,V1_1,V1_2,V1_3,V4_1,...,V5_1,V5_2,V5_3,V5_4,V6_1,V6_2,V6_3,V6_4,V7_1,V7_2
107,162,Yes,1,33.030562,-0.732194,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
328,503,Yes,1,31.760033,1.518828,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
352,534,Yes,1,17.512709,1.259095,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
383,580,Yes,1,31.043735,-1.338239,0,1,0,0,0,...,1,0,0,0,0,1,0,0,1,0
423,639,No,1,34.901993,-0.905350,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1


In [22]:
# Generate feature matrix x and reponse y
# Split them into train data and test data
x = sample.loc[:,'V2':]
y = sample['purchase']

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

In [23]:
# Use train_x and train_y to train the logistic model
lgr = LogisticRegression()

lgr.fit(train_x, train_y)

LogisticRegression()

In [24]:
# Put test_x into the model to predict pred_y
# Compute the precision rate and recall rate
pred_y = lgr.predict(test_x)

precision = precision_score(test_y, pred_y)
recall = recall_score(test_y, pred_y)

print('The precision score of the model is %.4f' %precision)
print('The recall score of the model is %.4f' %recall)

The precision score of the model is 0.5662
The recall score of the model is 0.5822


In [25]:
# use a confusion matrix to show the comparison of the predicted promotion and the actual promotion
con_mat = pd.DataFrame(confusion_matrix(pred_y, test_y), index = ['pred_No', 'pred_Yes'], columns = ['act_No', 'act_Yes'])
con_mat

,act_No,act_Yes
pred_No,108,89
pred_Yes,95,124


# Test the Promotion Strategy

In [26]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''

    x = pd.get_dummies(data = df, columns = cat_var)   
    
    #x = x.loc[:, 'V3':]
    promotion = lgr.predict(x)
    
    my_map = {0: "No", 1: "Yes"}
    promotion = np.array([my_map[i] for i in promotion])
    
    return promotion

In [27]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0180.

Your nir with this strategy is 279.70.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.01796459294878825, 279.70000000000005)